In [4]:
%load_ext kedro.ipython
%reload_kedro

[06/20/23 14:25:29] INFO     Resolved project path as:                                              ]8;id=446411;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=841183;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#136\136]8;;\
                             /Users/pchaberski/projects/llm-reading-assistant.                                     
                             To set a different path, run '%reload_kedro <project_root>'                           

                    WARNING  /Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/ ]8;id=428851;file:///Users/pchaberski/.pyenv/versions/3.10.10/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=462389;file:///Users/pchaberski/.pyenv/versions/3.10.10/lib/python3.10/warnings.py#109\109]8;;\
                             site-packages/kedro/framework/startup.py:98: DeprecationWarning:                      
                             project_version in pyproject.toml is deprecated, use                                  
                             kedro_init_version instead                                                            
                               warnings.warn(                                                                      
                                                                                                                   

[06/20/23 14:25:39] WARNING  /Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/ ]8;id=263881;file:///Users/pchaberski/.pyenv/versions/3.10.10/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=780;file:///Users/pchaberski/.pyenv/versions/3.10.10/lib/python3.10/warnings.py#109\109]8;;\
                             site-packages/kedro_mlflow/mlflow/kedro_pipeline_model.py:5:                          
                             DeprecationWarning: `kedro.extras.datasets` is deprecated and will be                 
                             removed in Kedro 0.19, install `kedro-datasets` instead by running                    
                             `pip install kedro-datasets`.                                                         
                               from kedro.extras.datasets.pickle import PickleDataSet                              
                                                                                                                   

2023/06/20 14:25:39 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/20 14:25:39 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 97727af70f4d -> 3500859a5d39, Add Model Aliases table
INFO  [alembic.runtime.migration] Running upgrade 3500859a5d39 -> 7f2a7d5fae7d, add datasets inputs input_tags tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [kedro.ipython] Kedro project llm-reading-assistant
INFO  [kedro.ipython] Defined global variable 'context', 'session', 'catalog' and 'pipelines'
WARNI [py.warnings] /Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/strawberry/types/fields/resolver.py:199: DeprecationWarning: Argument name-based matching of 'info' is

[06/20/23 14:25:42] INFO     Kedro project llm-reading-assistant                                    ]8;id=22673;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=40918;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#105\105]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=837655;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=236318;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#106\106]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=251411;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=770059;file:///Users/pchaberski/projects/llm-reading-assistant/.venv/lib/python3.10/site-packages/kedro/ipython/__init__.py#112\112]8;;\

In [1]:
from vertexai.preview.language_models import TextGenerationModel

In [2]:
def interview(temperature: float = .2) -> None:
    """Ideation example with a Large Language Model"""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 256,   
        "top_p": .8,                
        "top_k": 40,                 
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        'Give me ten interview questions for the role of program manager.',
        **parameters,
    )
    print(f"Response from Model: {response.text}")

In [3]:
interview()

Response from Model: 1. What is your experience with project management?
2. What is your process for managing a project?
3. How do you handle unexpected challenges or roadblocks?
4. How do you communicate with stakeholders and keep them updated on the progress of the project?
5. How do you measure the success of a project?
6. What are your strengths and weaknesses as a project manager?
7. What are your salary expectations?
8. What are your career goals?
9. Why are you interested in this position?
10. What questions do you have for me?


In [6]:
catalog.list()

['parameters',
 'params:prompt_prefixes',
 'params:prompt_prefixes.explain',
 'params:prompt_prefixes.summarize',
 'params:max_tokens',
 'params:max_tokens.explain',
 'params:max_tokens.summarize',
 'params:input_text',
 'params:mode',
 'params:model']

In [53]:
mode = "explain"
input_text = "London Interbank Offered Rate"
prompt_prefixes = context.params.get("prompt_prefixes")
max_tokens = context.params.get("max_tokens")
model = "text-bison@001"

In [56]:
from typing import Literal

def execute_prompt(
    mode: Literal["explain", "summarize"],
    input_text: str,
    prompt_prefixes: dict[str],
    max_tokens: dict[int],
    model: str = "text-davinci-003",
):
    """Construct a final prompt base on the selected `mode` and `input_text`.
    Execute prompt and return results.

    Args:
        mode (Literal[&quot;explain&quot;, &quot;summarize&quot;]): execution mode; either `explain` or `summarize`
        input_text (str): text to be explained or summarized
        prompt_prefixes (dict[str]): additional instructions for the model depending on the `mode`
        max_tokens (dict[int]): maximum number of tokens to generate depending on the `mode`
        model (str, optional): OpenAI model name. Defaults to "text-davinci-003".

    Returns:
        OpenAIObject: full response of the OpenAI model
    """
    prompt = prompt_prefixes[mode] + input_text
    # openai.api_key = _load_openai_api_key()

    # logger.info("Selected model: " + model)
    # logger.info("Input prompt: " + prompt)
    # logger.info("Executing prompt...")

    # response = openai.Completion.create(
    #     model=model,
    #     prompt=prompt,
    #     temperature=0,
    #     max_tokens=max_tokens[mode],
    #     top_p=1.0,
    #     frequency_penalty=0.0,
    #     presence_penalty=0.0,
    # )
    # logger.info("Done.")

    model = TextGenerationModel.from_pretrained(model)

    response = model.predict(
        prompt=prompt,
        max_output_tokens=max_tokens[mode],
        temperature=0.0, 
        top_p=.8,                
        top_k=40,    
    )

    return response

In [57]:
response = execute_prompt(
    mode,
    input_text,
    prompt_prefixes,
    max_tokens,
    model
)

In [59]:
type(response)

vertexai.language_models._language_models.TextGenerationResponse

In [60]:
response.text

'The London Interbank Offered Rate (LIBOR) is the average interest rate that banks charge each other for short-term loans. It is used as a benchmark for many financial products, such as mortgages and credit cards.'